This noteboook is designed as a prototype script for grabbing articles from the JSON file.

First let's create a small batch of the JSON objects.

In [97]:
pathway = "/Volumes/Seagate Backup Plus Drive 1/RC_2015-01"

In [192]:
import json
import shelve
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.append('[deleted]')
stopwords.append('deleted')

In [193]:
print stopwords

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

## Function to quickly make a python shelve database

In [100]:
def json_to_db(db_name,num_of_comments):
    

    '''
    Goes through the txt file, grabs the JSON objects in it.
    Uses shelve to create a database, with key= Subreddit and continually appending text to value.
    '''
    # open database
    d = shelve.open(db_name)
    
    # Set counter
    count = 0
    
    # Iterate through N JSON objects in file
    with open(pathway) as myfile:
        for item in myfile:
            if count < num_of_comments:
                
                # Load the JSON object
                json_object = json.loads(item)
                
                # Now append additional text to database
                if d.has_key(json_object['subreddit']):
                    
                    temp = d[json_object['subreddit']]      # extracts the copy
                    temp = temp+' '+json_object['body']     # mutates the copy
                    d[json_object['subreddit']] = temp      # stores the copy right back, to persist it

                # Or set the first entry for that subreddit
                else:
                    d[json_object['subreddit']] = json_object['body']
                
    
                count += 1
            else:
                d.close()
                break

In [101]:
json_to_db('10000_comments',10000)

In [102]:
d = shelve.open('10000_comments.db')

In [103]:
d['MakeupAddiction']

u"Thanks! Now on to NYE makeup! [deleted] Right!? My face loves me, my wallet hates me.  Simple and Tasteful. :) Dipbrow is definitely something that has to be learned haha. I just barely stick the end of my brush into the pot and thats usually enough to cover my whole brow. But I also use the brow wiz first to outline my brows.  They look great on you, especially the red. How much do these cost if you don't mind me asking? Yeah she did that in high school...I've also recently read that she lied about being molested. Not sure how I feel about her now especially since she hardly does beauty vids anymore. She def is lacking when it comes to effort in her vids. I mixed a little of a Tarte stain (berry colored) with the baby lips clear chapstick, I've had the Tarte stain forever and the label is worn off so i dont know the exact color but i'll try and find out! and thank you!! Seriously. The recommendations for Gerard cosmetics need to chill the fuck out. Sona Gasparian's no makeup-makeup 

## Tokenization and Cleaning Text

Here we focus on converting the large string into a list of words (tokenized and stopwords removed)

In [109]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# Output tokenizes text and removes any stopwords and then outptus lowercased words
output = [word.lower() for word in tokenizer.tokenize(d['MakeupAddiction']) if not word.lower() in stopwords]

In [113]:
len(tokenizer.tokenize(d['MakeupAddiction']))

722

In [119]:
len(output)

390

## Final word cleaner code is below

In [120]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def text_cleaner(text):
    '''
    INPUT: string of body text
    OUTPUT: List of tokenized lower case words with stopwords removed
    '''
    
    
    # Output tokenizes text and removes any stopwords and then outptus lowercased words
    return [word.lower() for word in tokenizer.tokenize(text) if not word.lower() in stopwords]
    
#text_cleaner(d['MakeupAddiction'])

# MONGO DB Setup

This focuses on setting up a MongoDB and Push the new body of comments to the collection

In [180]:
import pymongo

# Connect with db
#reddit_data = pymongo.Connection()['reddit_database']['comments']

def update_mongo(collection,subreddit,comment):
    '''
    INPUT: subreddit name and comment as a cleaned list of words
    RESULT: Updates MongoDB reddit_data database with $push
    '''
    
    # NOTE: This pushes the entire list, so final result is a list of comment lists
    # Use itertools.chain.from_iterable(reddit_data[subreddit]) to combine this for doc2vec
    collection.update({'subreddit': subreddit},
                      {'$push':{'body':comment}},upsert = True,
                      safe=True)


In [181]:
comment_test = text_cleaner(d['MakeupAddiction'])
update_mongo(reddit_data,'MakeupAddiction',comment_test)

## Clears database

In [172]:
def clear_mongo(mongodb):
    '''
    INPUT: A mongodb database
    RESULT: Based on user input, clears database.
    '''
    ans = raw_input('WARNING: THIS WILL CLEAR THE DATABASE. ARE YOU SURE? y/n')
    if ans == 'y':
        mongodb.remove()
        print 'Database has been cleared.'
    else:
        print 'Database not cleared.'
        
        
#clear_mongo(reddit_data)

## Take in SubReddit, return document list of words

In [174]:
import itertools 

def get_doc_list(subreddit):
    '''
    INPUT: Subreddit name
    OUTPUT: A document = list of words for that subreddit
    '''
    
    for obj in reddit_data.find({'subreddit':'MakeupAddiction'}):
        return list(itertools.chain.from_iterable(obj['body']))

## Create a MongoDB 

In [175]:
import pymongo

def create_or_connect(database,collection):
    '''
    INPUT: Database name (str) and collection (name)
    OUTPUT: Either creates the database or connects to it if it already exists.
            Returns the collections in the database cursor.
    '''
    
    return pymongo.Connection()[database][collection]

#create_or_connect('check','test_coll') works!

## Create MongoDB from my Reddit Data

In [182]:
# Clear the db first
clear_mongo(reddit_data)

Database has been cleared.


In [185]:
# Make or Connect to Database
reddit_data = create_or_connect('reddit_database','reddit_comments')

def reddit_data_pusher(db_name,num_of_comments):
    
    '''
    Goes through the txt file, grabs the JSON objects in it.
    Uses MongoDB to create a database, with key= Subreddit and continually appending clean text word list to value.
    '''
    
    # Set counter
    count = 0
    
    # Iterate through N JSON objects in file
    with open(pathway) as myfile:
        for item in myfile:
            if count < num_of_comments:
                
                # Load the JSON object
                json_object = json.loads(item)
                
                # Clean and tokenize text
                body = text_cleaner(json_object['body'])
                
                # $push to MongoDB
                update_mongo(reddit_data,json_object['subreddit'],body)
                
                # For safety puposes
                count += 1
            else:
                break

In [194]:
clear_mongo(reddit_data)
reddit_data = create_or_connect('reddit_database','reddit_comments')
reddit_data_pusher(reddit_data,100)

Database has been cleared.


In [196]:

for obj in reddit_data.find():
    print obj['subreddit']
    print obj['body']
    print '\n'

BMW
[[u'chappelle', u'https', u'www', u'youtube', u'com', u'watch', u'v', u'4b93boc9ylg']]


Smite
[[u'finish', u'masters', u'grind', u'1v1', u'joust'], [u'buy', u'everything', u'line', u'hi', u'rez', u'pockets', u'money']]


BabyBumps
[[u'thought', u'wanted', u'name', u'baby', u'primrose', u'like', u'hmm', u'hate']]


randomsuperpowers
[[u'slightly', u'stronger', u'rpg']]


Android
[[u'wish', u'google', u'dropped', u'support', u'android', u'lt', u'4', u'0', u'already']]


NSFW_GIF
[[u'made', u'multiscreen', u'video', u'sony', u'vegas', u'converted', u'webm']]


pokemontrades
[[u'please', u'take', u'modest', u'one']]


EverythingScience
[[u'm', u'going', u'leave', u'puff', u'erfish']]


MakeupAddiction
[[u'thanks', u'nye', u'makeup']]


Wishlist
[[u'wish']]


nba
[[u'thanks', u'making', u'much', u'better']]


RandomActsOfSTC
[[u'yes', u'please']]


techsupport
[[u'either', u'power', u'issue', u'dead', u'mainboard', u'suspect', u'short', u'mainboard', u'case']]


badkarma
[[u'http', u'i

In [88]:

# update the email address and the score at the same time 
# using $set in a single write. 
collection.update({"_id":"subreddit3"},
{"$push":{"value":['this','is','even','more',]}}, safe=True)


{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

In [ ]:
# testing

# insert an article and print out it's contents (make sure it's there)
articles.insert({"title": "tyler's article"})
article = articles.find_one()
#article
#{u'_id': ObjectId('4f0b353c096f762312000002'), u'title': u"tyler's article"}

# update the title to something new
article['title'] = "test"

# save it
articles.save(article)

# check to see if it changed
print articles.find_one()
#{u'_id': ObjectId('4f0b353c096f762312000002'), u'title': u'test'}

In [ ]:
from pymongo import Connection

# Example dictionary
d = {'greetings' : ['hi','hello'], 'askreddit' : ['what','who','why'], 'subreddit3' : ['text1','text2']}


#conn = Connection()
#db = conn['example-database']
#collection = db['example-collection']

for sub, body in d.items():
    collection.save({'_id' : sub, 'value' : body})
# testing
newD = {}
for obj in collection.find():
    newD[obj['_id']] = obj['value']
print newD
# output is: {u'string2': 2, u'string3': 3, u'string1': 1}